# 01. Linear Regression with TF

<p style="text-align: right;">
rayleigh@dplus.company<br>
Your name :
</p>

### 텐서플로 실습시간 그 두번째!<br>

#### 선행지식
1. 모델링을 한다는 것에 대한 이해.
2. Supervised Learning 중 Regression에 대한 기본 지식.
3. 리니어 리그레션에 대한 기본 구조와 개념
4. 앞서 진행한 실습에 대한 추억(00. TensorFlow Basics, 01. Linear Regression with TF)

#### 실습목표
1. Linear Regression의 구조를 Graph로 그려낼 수 있다.
2. tf.Variable과 초기화, Optimizer를 사용할 수 있다.
3. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.
4. (Optional) Statistical Model의 Signal & Ratio 를 이해한다.
5. (Optional) Encoding이 무엇인지 이해한다. --> Dummy Variable, Onehot-encoding 등의 필요성을 이해한다.


#### 사용데이터.

01. Univariate Linear Regression : Lemonade.csv (가상 데이터셋)
02. Multivariate Linear Regression : http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset

# Univariate Linear Regression

## 00. 라이브러리 불러오기

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import os

## 01. 데이터 불러오기 & 문제정의 & 전처리

여러분은 이동식 레모네이드 카페, '롤로노아'의 사장입니다.<br>
차별화된 맛과, 원재료의 신선함. 한 번도 안먹어본 사람은 있을 수 있어도, 한 번만 마셔본 사람은 없다!<br>
수년간 늦봄~초가을에 포장마차를 끌고 행복을 팔며 느꼈던 것은...<br>
#### 여러분의 불행은 팔지못하고 남은 재료의 양과 비례한다는 것입니다.<br>

갈기 편하게 미리 손질해둔 레몬은 그날이 지나면 더 이상 갈아서 팔 수가 없습니다..<br>
물론, 하루이틀쯤 지났다고 해서 맛이 크게 변할리 없고 냉장고 안에서 쉽게 상할리 없지만, 분명 퀄리티가 다릅니다.<br>
오늘도 당신은, 눈물을 흘리며 레몬을 버립니다...

In [ ]:
data = pd.read_csv('Lemonade.csv')
print(data.shape)
data.head(n=10)

In [ ]:
# Pandas Dataframe to Numpy Array
np_data = data.values

# Shuffle
np.random.shuffle(np_data)

# Split
train, test = np_data[:1600], np_data[1600:]

X_train, Y_train = train[:, 0].reshape(-1, 1), train[:, 1].reshape(-1, 1)
X_test, Y_test = test[:, 0].reshape(-1, 1), test[:, 1].reshape(-1, 1)

In [ ]:
# 20개의 데이터 산점도
data[:20].plot.scatter(x='Temperature', y='lemonade')

## 02. 손으로 그려보는 Linear Regression



*직접, 손으로 구조를 설계 해보는 것은 중요하다.*<br>

여러분이 텐서플로우로 '그래프'를 코딩하기에 최대한 편한 설계도를 그리면 된다.<br>
꼭 Data flow Graph, Computation Grpah, Probabilistic Graph 등의 표현을 따를 필요는 없다.(물론, 익히면 좋다.)<br>
#### 이번엔 같이 그려보자!
##### 하나하나 이해하면서 다같이 그려보자!

![regression](img/regression_0.png)

#### 차원을 반드시 고려 해야한다.

In [ ]:
print(X_train.shape, Y_train.shape)

## 03. Make a Graph with TF

In [ ]:
'''
print(X_train.shape, Y_train.shape)
(1600, 1),   (1600, 1)
'''
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01

# Placeholders
X = tf.placeholder(dtype=tf.float32, shape=[None, 1], name='X_input')
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1], name='Y_input')

# Weight & Bias
W = tf.Variable(tf.random_normal([1, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
Y_pred = tf.add(tf.matmul(X, W), b, name='Y_predicted')

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.squared_difference(Y, Y_pred), name='MSE')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


## 04. Train! Session!

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

learning_rate = 1

start = time.clock()
for i in range(2000):
    # 학습 진행
    sess.run(optimizer, feed_dict={X: X_train, Y: Y_train})
    
    if (i % 100 == 0):
        # 학습 상황 디스플레이
        tr_l = sess.run(cost, feed_dict={X: X_train, Y: Y_train})
        print("{}th training---- training loss: {:.2f}".format(i, tr_l))

print("Training Complete")
end = time.clock()

print("Total elapsed time: {}".format(end - start))
print("Test loss: {:.2f}".format(sess.run(cost, feed_dict={X: X_test, Y: Y_test})))

Y_predicted = sess.run(Y_pred, feed_dict={X: X_test, Y: Y_test})

#### 데이터의 인풋은 온도였습니다. 아웃풋은 판매량이었구요.

아래의 내용이 해석이 되어야 합니다.

In [ ]:
print(sess.run([W, b]))

In [ ]:
# 세션을 닫아야 합니다.
sess.close()

### Check

In [ ]:
temp = pd.DataFrame({'Y_real': Y_test.reshape(-1, ), 'Y_predicted': Y_predicted.reshape(-1, )})
temp.plot.scatter(x='Y_real', y='Y_predicted', figsize=(5, 5))

In [ ]:
%reset()

# Multivariate Linear Regression

## 00. 라이브러리 불러오기

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import os

# 현재 작업 디렉토리 확인
os.getcwd()

## 01. 데이터 불러오기 & 문제정의 & 전처리

In [ ]:
rides = pd.read_csv('./Bike-Sharing-Dataset/hour.csv')
print(rides.shape)
rides.head(n=3)

#### 데이터 설명.

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return back has become automatic.<br>
(구글번역) 자전거 공유 시스템은 멤버쉽, 렌탈 및 리턴 백의 전체 프로세스가 자동으로 이루어지는 전통적인 자전거 렌탈의 차세대 제품입니다.<br>

[들어가서 Data Set Information을 읽어보자.](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)

[ 무언가를 위한 어떤 필요!! ]
의 첫 출발로 자전거를 빌려가는 양에 대한 예측을 진행할 것이다.

In [ ]:
# 일부의 데이터를 그래프로 확인.
rides[:24*10].plot(x='dteday', y='cnt')

In [ ]:
rides[:24*10].plot.scatter(x='atemp', y='cnt')

### Dummy Variable & Drop

데이터셋 안에 '국가'라는 컬럼이 있다고 생각해보자.<br>
그걸 그대로 사용할 때 펼쳐질 에러에 대해서 논해보자.

In [ ]:
# 더비 변수 생성, one hot encoding
dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)

# 필요없는 칼럼 제거
fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 'mnth', 'hr', 'weekday']
data = rides.drop(fields_to_drop, axis=1)
data.head()

### Scaling

In [ ]:
# scaling 이전의 데이터
data[['casual', 'registered', 'cnt', 'temp', 'atemp', 'hum', 'windspeed']].head(n=10)

In [ ]:
quant_features = ['casual', 'registered', 'cnt', 'temp', 'atemp', 'hum', 'windspeed']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean) / std
    
data[['casual', 'registered', 'cnt', 'temp', 'atemp', 'hum', 'windspeed']].head(n=10)

### Splitting the data into training, testing sets

맨 마지막의 61일을 test_set으로, 나머지를 Training set으로 둔다.<br>
"일단은 익숙해지자. "

In [ ]:
train_data, test_data = data[:-61*24], data[-61*24:]

target_fields = ['cnt', 'casual', 'registered']
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]
train_features, train_targets = train_data.drop(target_fields, axis=1), train_data[target_fields]

train_targets.head()

In [ ]:
test_X, test_Y = test_features.values, test_targets.values
train_X, train_Y = train_features.values, train_targets.values

print(train_X, train_Y)

In [ ]:
print(train_X.shape, train_Y.shape)

## 02. 손으로 그려보는 Linear Regression



*직접, 손으로 구조를 설계 해보는 것은 중요하다.*<br>

여러분이 텐서플로우로 '그래프'를 코딩하기에 최대한 편한 설계도를 그리면 된다.<br>
꼭 Data flow Graph, Computation Grpah, Probabilistic Graph 등의 표현을 따를 필요는 없다.(물론, 익히면 좋다.)<br>

### 예시.  여러분이 직접 그려보기 전에는 실행 시키지 말기!

[리그레션01](img/regression_1.PNG)
[리그레션02](img/regression_2.PNG)

### 반드시 차원을 고려 할 것!

X의 shape = [ 채워넣기! , 채워넣기! ]

Y의 shape = [ 채워넣기! , 채워넣기! ]

W의 shape = [ 채워넣기! , 채워넣기! ]

bias의 shape = [ 채워넣기! , 채워넣기! ]

연산순서는?

## 03. Make a Graph with TF

In [ ]:
'''
여러분이 직접 스크립트를 짜야 합니다.
조금 힌트는 드립니다..!

print(train_X.shape, train_Y.shape)
(15435, 58),   (15435, 1)
'''
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01

# Placeholders
X = tf.placeholder(dtype=tf.float32, shape=[None, 58], name='X_input')
Y = tf.placeholder(dtype=tf.float32, shape=[None, 3], name='Y_input')

# Weight & Bias
W = tf.Variable(tf.random_normal([58, 3]), name='weight')
b = tf.Variable(tf.random_normal([3]), name='bias')

# Hypothesis
Y_pred = tf.add(tf.matmul(X, W), b, name='Y_predicted')

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.squared_difference(Y, Y_pred), name='MSE')
Optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


## 04. Train! Session!

In [ ]:
'''
Weight, bias 초기화를 먼저 해야함.

초기화란? :
'''
sess = tf.Session()
sess.run(tf.global_variables_initializer())

learning_rate = 0.01

start = time.clock()
for i in range(1000):
    # 학습 진행
    sess.run(Optimizer, feed_dict={X: train_X, Y: train_Y})
    
    if (i % 100 == 0):
        # 학습 상황 디스플레이
        tr_l = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
        print("{}th training---- training loss: {:.2f}".format(i, tr_l))
        
print("Training Complete")
end = time.clock()

print("Total elapsed time: {}".format(end - start))
print("Test loss: {:.2f}".format(sess.run(cost, feed_dict={X: test_X, Y: test_Y})))

Y_predicted = sess.run(Y_pred, feed_dict={X: test_X, Y: test_Y})

In [ ]:
weight, bias = sess.run([W, b])
print(weight)
print(bias)

In [ ]:
# 세션을 닫아야 합니다.
sess.close()

### Check Out on Test set

In [ ]:
# target_fields = ['cnt', 'casual', 'registered']
column = 'cnt'
cindex = target_fields.index(column)
rows = 100

# 예측값도 Scaling 되어 있으므로 그것을 원래대로 되돌려주는 과정
mean, std = scaled_features[column]
simple_predictions = Y_predicted[:rows, cindex] * std + mean
simple_predictions


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(simple_predictions, label='Simple_Prediction')
ax.plot(test_Y[:rows, cindex] * std + mean, label='Real')
ax.set_xlim(right=len(simple_predictions))
ax.legend()

dates = pd.to_datetime(rides.loc[test_data[:rows].index]['dteday'])
dates = dates.apply(lambda d: d.strftime('%b %d'))
ax.set_xticks(np.arange(len(dates))[12::24])
_ = ax.set_xticklabels(dates[12::24], rotation=45)

# must Summary!